### Pitching Analysis

Import the necessary statements and set the dataframes to show all the columns.

In [29]:
import pandas as pd
import pickle
pd.set_option('display.max_columns', None)

Uncomment this line to get the most recent statistics from the DakStats website.

In [30]:
#%run ./Conference_Statistics.ipynb

We import our dataframes for batting, pitching, and fielding statistics as well as our list of teams from the pickle file titled `Stats.pkl`.

In [31]:
with open('Stats.pkl', 'rb') as f:
    dfb = pickle.load(f)
    dfp = pickle.load(f)
    dff = pickle.load(f)
    teams = pickle.load(f)

To gather our totals for each team we take only the total row for each team and rename the dataframe `pitch_totals`. We also delete the `GS` column as the number of games started is not applicable for team totals, only individuals.

In [32]:
pitch_totals = [df[df.Pitching.str.contains("Total:", regex = False)] for df in dfp]
for df in pitch_totals:
    del df["GS"]
pitch_totals[2]

,Pitching,ERA,W,L,GP,CG,SHO,CBO,SV,IP,H,R,ER,BB,SO,2B,3B,HR,TBF,B_AVG,WP,HBP,BK,SFA,SHA
12,Total:,5.87,6,18,24,3,0,0,1,185.2,225,164,121,73,151,35,7,14,855,0.301,22,20,6,13,2


We combine these total rows into a single dataframe named `merged_pitch_totals`.

In [33]:
merged_pitch_totals = pd.concat(pitch_totals)
merged_pitch_totals[:4]

,Pitching,ERA,W,L,GP,CG,SHO,CBO,SV,IP,H,R,ER,BB,SO,2B,3B,HR,TBF,B_AVG,WP,HBP,BK,SFA,SHA
15,Total:,5.43,6,18,24,7,2,0,1,185.2,200,153,112,77,127,44,1,20,854,0.273,18,25,0,11,9
17,Total:,6.95,2,22,24,3,0,0,2,177.1,219,200,137,108,149,47,8,17,924,0.298,26,50,4,16,15
12,Total:,5.87,6,18,24,3,0,0,1,185.2,225,164,121,73,151,35,7,14,855,0.301,22,20,6,13,2
9,Total:,3.43,17,7,24,6,1,0,5,191.2,176,81,73,71,149,20,2,14,830,0.239,12,15,3,2,5


For those familiar with baseball, the interpretation of the `IP` column displaying the number of innings pitched by each player is fairly straightforward. However, the decimals are not mathematically equivalent to their numerical value. Because each inning consists of three outs, if a pitcher gets only one out in the inning, their `IP` total would display `0.1` even though mathematically, they threw 1/3 or 0.333... of an inning. To do our calculations we must fix this before summing the total innings pitched. First, we gather the innings pitched in an individual list named `ips`.

In [34]:
ips = merged_pitch_totals["IP"].to_list()

We then run a test for each number in our list. We multiply the number of innings by 10 to eliminate the decimal and then compare the modulus 10 of each number. If that number mod 10 is equal to 0, we know the innings pitched is a whole number and we do nothing. If the number mod 10 is equal to 1, we know that one extra out was earned and we correct the number to 0.333 rather than 0.1 by adding 0.233. Similarly, if the number mod 10 is equal to 2, we must add 0.467 to acheive a decimal of 0.667. Once we have changed each our innings, we copy the dataframe again to avoid any errors and reset the `IP` column to the values in the list `ips`.

In [35]:
for i in range(len(ips)):
    if ((ips[i]*10) % 10) == 0:
        pass
    elif ((ips[i]*10) % 10) == 1:
        ips[i] = ips[i] + 0.233
    elif ((ips[i]*10) % 10) == 2:
        ips[i] = ips[i] + 0.467
fixed_ip_totals = merged_pitch_totals.copy()
fixed_ip_totals["IP"] = ips
fixed_ip_totals[:4]

,Pitching,ERA,W,L,GP,CG,SHO,CBO,SV,IP,H,R,ER,BB,SO,2B,3B,HR,TBF,B_AVG,WP,HBP,BK,SFA,SHA
15,Total:,5.43,6,18,24,7,2,0,1,185.667,200,153,112,77,127,44,1,20,854,0.273,18,25,0,11,9
17,Total:,6.95,2,22,24,3,0,0,2,177.333,219,200,137,108,149,47,8,17,924,0.298,26,50,4,16,15
12,Total:,5.87,6,18,24,3,0,0,1,185.667,225,164,121,73,151,35,7,14,855,0.301,22,20,6,13,2
9,Total:,3.43,17,7,24,6,1,0,5,191.667,176,81,73,71,149,20,2,14,830,0.239,12,15,3,2,5


We add a total row at the bottom with the code `.sum()` and we delete the `Pitching` column as it does not hold any meaningful information. Finally, we subset only the last column that contains the final totals with the code `.iloc[-1:]`.

In [36]:
fixed_ip_totals.loc["CL_Total"] = fixed_ip_totals.sum()
del fixed_ip_totals["Pitching"]
CL_pitch_totals = fixed_ip_totals.iloc[-1,:]

Since some of the statistics are averages, we have to go back and calculate those by hand because the summation incorrectly calculates those metrics. A few of the conference totals are printed.

In [37]:
CL_tot_p = CL_pitch_totals.copy()
#fix opponent batting average
CL_tot_p["B_AVG"] = round(CL_tot_p["H"] / (CL_tot_p["TBF"] - CL_tot_p["BB"] - CL_tot_p["HBP"] - CL_tot_p["SFA"] - CL_tot_p["SHA"]), 3)
#fix ERA
CL_tot_p["ERA"] = round((CL_tot_p["ER"]*27) / round(CL_tot_p["IP"]*3), 3)

In [38]:
CL_tot_p[:5]

ERA      4.989
W      120.000
L      119.000
GP     240.000
CG      34.000
Name: CL_Total, dtype: float64

Next, in order to rank the Crossroads League pitchers, we will calculate Fielding Independent Pitching (FIP), a statistic that measures a pitcher's effectiveness regardless of defense or inherent luck in hitting. Our first step is to calculate the league constant, which is shown below.

In [39]:
CL_FIP_c = CL_tot_p["ERA"] - (((13*CL_tot_p["HR"] + 3*(CL_tot_p["BB"] + CL_tot_p["HBP"]) - 2*CL_tot_p["SO"])*3) / round(CL_tot_p["IP"]*3))
CL_FIP_c

4.000357490535425

Just like we did with the totals dataframe, we must correct for the notation used in the `IP` column for each team's individual statistical table.

In [40]:
for df in dfp:
    ips = df["IP"].to_list()
    for i in range(len(df.index)):
        if ((ips[i]*10) % 10) == 0:
            pass
        elif ((ips[i]*10) % 10) == 1:
            ips[i] = ips[i] + 0.233
        elif ((ips[i]*10) % 10) == 2:
            ips[i] = ips[i] + 0.467
    df["IP"] = ips

Now that we have workable data, we can calculate our individual FIP statistics for each pitcher. We also add a column for each player's team to improve readability in our final rankings. After creating the column, we move it up to be the first statistical column displayed.

In [41]:
for i in range(len(teams)): #add column for team
    dfp[i]["Team"] = teams[i]
for df in dfp:
    df["FIP"] = round(((13*df["HR"] + 3*(df["BB"] + df["HBP"]) - 2*df["SO"])*3) / round(df["IP"]*3) + CL_FIP_c, 3)
    df["WHIP+"] = round((df["BB"]+df["HBP"]+df["H"])/df["IP"], 2)
    team = df.pop("Team")
    df.insert(1, team.name, team) #move team column to second
    fip = df.pop("FIP")
    df.insert(2, fip.name, fip) #move FIP column to third

We tidy up the data by removing the totals and the opponents rows and removing the unnecessary decimal place in the `GS` column.

In [42]:
for df in dfp:
    df.drop(df.tail(2).index,inplace=True) # drop last 2 rows(only run this code once otherwise data will be lost)
    df['GS'] = df['GS'].astype(int) #remove decimal place on GS column
dfp[2][-2:]

,Pitching,Team,FIP,ERA,W,L,GP,GS,CG,SHO,CBO,SV,IP,H,R,ER,BB,SO,2B,3B,HR,TBF,B_AVG,WP,HBP,BK,SFA,SHA,WHIP+
10,"Platt, Zach",Grace,11.667,15.00,0,0,2,0,0,0,0,0,3.000,8,13,5,4,4,1,0,1,25,0.421,1,2,0,0,0,4.67
11,"Bertke, Ryan",Grace,6.728,19.64,0,0,2,1,0,0,0,0,3.667,8,8,8,4,4,0,0,0,24,0.444,2,2,1,0,0,3.82


We subset the data to only include pitchers with at least 10 conference innings to ensure the best pitchers are recognized.

In [43]:
#make sure everyone has at least 10 IP
for i in range(len(teams)):
    dfp[i] = dfp[i][dfp[i]['IP'] >= 10]

Finally, we combine all of the dataframes with the code `pd.concat`. Then, we sort the rows by FIP beginning with the lowest. To maintain the standard notation of the `IP` column, we run a test to check for pitchers with innings pitched ending in 0.333 or 0.667 and change them back to 0.1 and 0.2 respectively. The top ten Crossroads League pitchers in terms of FIP are shown below.

In [44]:
all_pitchers = pd.concat(dfp)
top_pitchers = all_pitchers.sort_values(by=['FIP']) #sort by FIP in ascending order order
ips = top_pitchers["IP"].to_list() #change IP back to standard notation
for i in range(len(top_pitchers.index)):
    if ((ips[i]*3) % 3) == 0:
        pass
    elif (round(ips[i]*3) % 3) == 1:
        ips[i] = ips[i] - 0.233
    elif (round(ips[i]*3) % 3) == 2:
        ips[i] = ips[i] - 0.467
top_pitchers["IP"] = ips
top_pitchers[:20]

,Pitching,Team,FIP,ERA,W,L,GP,GS,CG,SHO,CBO,SV,IP,H,R,ER,BB,SO,2B,3B,HR,TBF,B_AVG,WP,HBP,BK,SFA,SHA,WHIP+
4,"Noelker, Andrew",Marian,2.435,4.11,0,2,5,3,1,0,0,0,15.1,12,10,7,5,26,3,0,1,63,0.211,2,0,0,0,1,1.11
5,"Ubelhor, Mitch",Taylor,2.596,2.87,0,1,8,1,0,0,1,1,15.2,9,8,5,8,26,3,0,0,67,0.158,3,2,0,0,0,1.21
1,"Hoffman, Hunter",IWU,2.707,0.93,3,0,6,6,1,1,0,0,38.2,21,7,4,15,49,4,0,0,150,0.157,1,1,0,0,0,0.96
9,"Gongwer, Drake",Taylor,2.841,4.30,1,0,6,0,0,0,0,0,14.2,11,9,7,3,24,5,0,1,60,0.208,1,3,1,0,0,1.16
4,"Huseman, Noah",Taylor,3.050,1.87,5,1,6,6,1,1,1,0,33.2,21,11,7,13,43,2,1,0,140,0.175,2,5,1,0,2,1.16
2,"Norman, Braedon",IWU,3.056,1.50,2,1,7,1,0,0,1,0,18.0,13,4,3,4,16,5,0,0,72,0.197,2,1,0,0,1,1.00
13,"Kraabel, Zach",SAU,3.110,7.59,2,2,6,5,0,0,0,0,21.1,32,20,18,8,23,14,1,0,105,0.348,2,1,0,2,2,1.92
1,"Mccutcheon, Alex",HU,3.346,1.96,4,2,6,6,3,1,0,0,36.2,26,11,8,9,35,2,2,1,147,0.193,1,2,1,1,0,1.01
8,"Moran, Joe",Taylor,3.364,3.82,3,1,6,6,0,0,1,0,33.0,32,18,14,8,45,6,0,3,143,0.248,2,2,0,1,3,1.27
2,"Shinabery, Mason",HU,3.384,2.41,4,1,6,5,2,0,0,1,37.1,27,10,10,6,35,8,0,2,144,0.197,3,1,0,0,0,0.91


In [45]:
sum(top_pitchers["FIP"])/len(top_pitchers)

4.932227272727272

In [46]:
dfp[2]

,Pitching,Team,FIP,ERA,W,L,GP,GS,CG,SHO,CBO,SV,IP,H,R,ER,BB,SO,2B,3B,HR,TBF,B_AVG,WP,HBP,BK,SFA,SHA,WHIP+
3,"Koch, Kameron",Grace,3.790,3.55,2,2,6,6,2,0,0,0,38.000,41,17,15,13,30,7,2,1,165,0.275,5,0,0,2,1,1.42
4,"Decker, Jake",Grace,5.388,5.06,0,0,6,2,0,0,0,0,26.667,24,15,15,9,21,2,3,4,108,0.247,2,0,0,2,0,1.24
5,"Haney, Houston",Grace,4.106,5.68,1,3,6,6,1,0,0,0,38.000,49,30,24,5,26,10,1,2,167,0.318,1,5,0,2,1,1.55
6,"Etchison, Evan",Grace,5.138,6.05,1,2,6,3,0,0,0,1,19.333,19,16,13,6,19,4,0,3,83,0.264,3,1,2,4,0,1.34
7,"Schumacher, Hunter",Grace,4.706,6.35,0,2,5,1,0,0,0,0,11.333,17,13,8,6,11,7,1,0,60,0.340,1,4,1,0,0,2.38
8,"Meyer, Mason",Grace,6.095,7.13,1,2,6,2,0,0,0,0,17.667,22,20,14,12,12,1,0,1,87,0.310,4,4,0,0,0,2.15
9,"Clark, Tanner",Grace,6.546,11.45,0,3,5,3,0,0,0,0,11.000,19,19,14,8,6,1,0,1,62,0.365,1,1,0,1,0,2.55
